# Bibliotecas

In [1]:
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import requests
import json
import time
import pandas as pd
import numpy as np

# Funções

In [125]:
def rasp(site):
    site = site
    
    answer = requests.get(site)
    answer = answer.content

    soup_ = BeautifulSoup(answer)

    list_ = soup_.find_all('a', {'class':'heading'})
    
    dom_json = []
    title_json = []

    count = 0

    for i in list_:
        if count%2 == 1:
            dom_json.append(i.get('href'))
            title_json.append(i.get('title'))
        count += 1

    title_json.append('Todas as tabelas')

    if selection.value == 'Todas':
        pass
    else:
        option_table = widgets.Dropdown(options = title_json,
                                       value = title_json[0],
                                       description = 'Dados: ')
        display(option_table)
    if selection.value == 'Todas':
        return title_json, dom_json
    else:
        return option_table, title_json, dom_json

def rasp_table(title, dom, index):
    title_json = title
    dom_json = dom
    new_answer = requests.get('http://dados.gov.br' + str(dom_json[index]))
    new_answer = new_answer.content
    soup_1 = BeautifulSoup(new_answer)
    tab1 = soup_1.find_all('a')
    api_url = []
    for i in tab1:
        if ('aneel' and 'Tarifa') in str(i.get('href')):
            api_url.append(i.get('href'))
    new_answer1 = requests.get(str(api_url[1]))
    data = json.loads(str(new_answer1.text))
    df = pd.DataFrame(data)
    df.to_excel('D:\\Dados dos Usuários\\priscila.miyashita\\Downloads\\' + str(title_json[index]) + '.xlsx', index_label = None, encoding = 'UTF-8')
    
def if_(selection, title, dom):
    try:
        if option.value != 'Todas as tabelas':
            count = 0
            for i in title:
                if option.value == i:
                    index = count
                count += 1
            rasp_table(title, dom, index)
        else:
            w = widgets.IntProgress(value=0,min=1,max=len(dom),step=1,description='Carregando:',bar_style='info', orientation='horizontal')
            display(w)
            for k in range(len(dom)):
                rasp_table(title, dom, k)
                w.value = k+1
    except:
        w = widgets.IntProgress(value=0,min=1,max=len(dom),step=1,description='Carregando:',bar_style='info', orientation='horizontal')
        for k in range(len(dom)):
            rasp_table(title, dom, k)
            w.value = k+1
            
def limpeza_html(lista, name):
    for i in str(lista[0]).split('<'):
        if name in i:
            return i.split('>')[1]

# Raspagem

In [116]:
site = 'http://dados.gov.br/dataset?tags=Tarifas+de+Energia+El%C3%A9trica'

resposta = requests.get(site)
resposta = resposta.content

soup_ = BeautifulSoup(resposta)
tab1 = soup_.find_all('a')
title_ = []
dom_ = []

for i in tab1:
    if 'dataset/tarifas' in str(i.get('href')):
        dom_.append(i.get('href'))
    
title_ = []
dom_ = np.asarray(dom_)
dom_ = np.unique(dom_)

for i in dom_:
    if 'tarifas' in i:
        title_.append((i.replace('/dataset/', '')).replace('-', ' ').capitalize())

title_.append('Todas')

option = widgets.Dropdown(options = title_, description = 'Tarifas de energia: ')
option

Dropdown(description='Tarifas de energia: ', options=('Tarifas medias de energia eletrica por classe de consum…

## Acessando a API e salvando o arquivo

In [139]:
if option.value != 'Todas':
    count = 0
    for i in title_:
        if i == option.value:
            break
    count += 1
    site = 'http://dados.gov.br' + str(dom_[count])
    if_(option.value, title_, dom_)
else:
    for i in range(len(title_)-1):
        site = 'http://dados.gov.br' + str(dom_[i])
        if_(title_[i], title_, dom_)